# Mask a variable on pressure levels where data are below surface<a id='top' class="tocSkip"> </a>

This tutorial shows how to mask data

In some case earth system model output are interpolated to som standard pressure levels, this can lead to data being generated bellow the ground

This tutorial shows how to mask these erroneous data.  Data courtesy of Jerry Potter

© The CDAT software was developed by LLNL. This tutorial was written by Charles Doutriaux. This work was performed under the auspices of the U.S. Department of Energy by Lawrence Livermore National Laboratory under Contract DE-AC52-07NA27344.

[Download the Jupyter Notebook](Mask_variable_on_pressure_where_data_are_bellow_the_surface.ipynb)



<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Prepare-Notebook" data-toc-modified-id="Prepare-Notebook-1">Prepare Notebook</a></span></li><li><span><a href="#Load-the-data" data-toc-modified-id="Load-the-data-2">Load the data</a></span></li><li><span><a href="#Regrid-surface-pressure-data-to-target-grid" data-toc-modified-id="Regrid-surface-pressure-data-to-target-grid-3">Regrid surface pressure data to target grid</a></span></li><li><span><a href="#Mask-Data-Below-Surface" data-toc-modified-id="Mask-Data-Below-Surface-4">Mask Data Below Surface</a></span></li><li><span><a href="#Plot-Data" data-toc-modified-id="Plot-Data-5">Plot Data</a></span></li></ul></div>

## Prepare Notebook
[Back to Top](#top)

In [ ]:
from __future__ import print_function
import cdms2
import numpy
import vcs
import requests
import os

for filename in ['ps.nc','ta.nc']:
    if not os.path.exists(filename):
        r = requests.get("https://cdat.llnl.gov/cdat/sample_data/notebooks/{}".format(filename), stream=True)
        with open(filename,"wb") as f:
            for chunk in r.iter_content(chunk_size=1024):
                if chunk:  # filter local_filename keep-alive new chunks
                    f.write(chunk)


## Load the data
[Back to Top](#top)

In [ ]:
PS_file = cdms2.open("ps.nc")
TA_file = cdms2.open("ta.nc")

ps = PS_file("ps")
ta = TA_file("ta")

print("Shaped: ps {} and ta {}".format(ps.shape,ta.shape))

## Regrid surface pressure data to target grid
[Back to Top](#top)

In [ ]:
ps = ps.regrid(ta.getGrid())
print("ps new shape:",ps.shape)

## Mask Data Below Surface
[Back to Top](#top)

In [ ]:
# Loop through levels and mask where pressure is less than ps

levels = ta.getLevel()
print("Levels are:", levels[:])
for i,level in enumerate(levels):
    low = numpy.less(ps,level)
    ta[:,i] = numpy.ma.masked_where(low,ta[:,i])

## Plot Data
[Back to Top](#top)

In [ ]:
x = vcs.init(bg=True,geometry=(600,400))
x.plot(ta[:,0])